In [234]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import re
import torch
import torch.nn as nn
import os
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset
import torchvision.models as models
from collections import defaultdict
from scipy.stats import ttest_rel, wilcoxon


In [235]:
import pandas as pd

# Replace 'your_file.tsv' with the path to your .tsv file
df = pd.read_csv('./data/filtered_results.tsv', sep='\t')

# Display the first few rows
print(df.head())


/var/folders/y8/q2s37ndx6tg3lpzp3vp8xk_r0000gn/T/ipykernel_12534/2273208280.py:4: DtypeWarning: Columns (40,44,46,50,54,58,94,100,104,110,118,124,126,128,130,132,134,136,140,142,146,148,150,152,154,156,158,178,186,192,198,202,214,218,230,236,238,240,244,246,248,250,252,254,256,258,260,262,264,266,270,272,274,276,286,288,290,292,294,296,298,300,302,304,306,308,310,312,314,316,318,320,324,326,332,334,336,338,340,342,344,346,348,352,366,374,376,380,382,384,390,392,394,396,400,406,408,410,412,414,416,418,428,430,442,446,448,456,460,462,466,468,470,486,504,512,516,518,524,526,528,530,540,542,544,546,554,556,558,560,564,578,594,608,610,612,614,620,624,626,628,630,632,634,642,644,646,648,652,654,656,658,660,662,664,666,676,678,680,686,688,694,696,698,700,702,704) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/filtered_results.tsv', sep='\t')


   rowid  ccms_row_id                   Peptidoform  \
0      1            1              .SPLFM+15.995GK.   
1      2            2     .EPQVYTLPPSREEM+15.995TK.   
2      3            3         .AVM+15.995DDFAAFVEK.   
3      4            4  .EFNAETFTFHADIC-33.988TLSEK.   
4      5            5   .M+15.995ADEAGSEADHEGTHSTK.   

               Peptidoform ID      Unmod peptidoform  Total  \
0              SPLFM+15.995GK              .SPLFMGK.  11679   
1     EPQVYTLPPSREEM+15.995TK     .EPQVYTLPPSREEMTK.  11411   
2         AVM+15.995DDFAAFVEK         .AVMDDFAAFVEK.  10949   
3  EFNAETFTFHADIC-33.988TLSEK  .EFNAETFTFHADICTLSEK.  10198   
4   M+15.995ADEAGSEADHEGTHSTK   .MADEAGSEADHEGTHSTK.   9608   

   Total- Unmodified sequence  Peptidoforms- Unmodified sequence  \
0                       11681                                  2   
1                       13133                                 46   
2                       18495                                 81   
3                 

In [236]:
print(df.columns)

Index(['rowid', 'ccms_row_id', 'Peptidoform', 'Peptidoform ID',
       'Unmod peptidoform', 'Total', 'Total- Unmodified sequence',
       'Peptidoforms- Unmodified sequence', 'Proteins', 'Mass',
       ...
       '_dyn_#Patient_M2_healthyMale.Timepoint_2_unmod',
       '_dyn_#Patient_M2_healthyMale.Timepoint_3',
       '_dyn_#Patient_M2_healthyMale.Timepoint_3_unmod',
       '_dyn_#Patient_M3_healthyMale.Timepoint_1',
       '_dyn_#Patient_M3_healthyMale.Timepoint_1_unmod',
       '_dyn_#Patient_M3_healthyMale.Timepoint_2',
       '_dyn_#Patient_M3_healthyMale.Timepoint_2_unmod',
       '_dyn_#Patient_M3_healthyMale.Timepoint_3',
       '_dyn_#Patient_M3_healthyMale.Timepoint_3_unmod', 'id'],
      dtype='object', length=707)


In [237]:
df.shape

(40921, 707)

In [238]:
threshold = 0.9 * df.shape[1]

num_rows_with_60_percent_null = (df.isnull().sum(axis=1) >= threshold).sum()
num_rows_with_60_percent_null

np.int64(26804)

In [239]:
pattern = r'unmod'
patient_timepoints = []
intensity_cols = [col for col in df.columns if re.search(pattern, col)]
intensity_cols

['Outlier groups- unmod',
 'Outlier group ratio- unmod',
 '_dyn_#Patient_01.Timepoint_1_unmod',
 '_dyn_#Patient_01.Timepoint_2_unmod',
 '_dyn_#Patient_01.Timepoint_3_unmod',
 '_dyn_#Patient_01.Timepoint_4_unmod',
 '_dyn_#Patient_01.Timepoint_5_unmod',
 '_dyn_#Patient_01.Timepoint_6_unmod',
 '_dyn_#Patient_01.Timepoint_7_unmod',
 '_dyn_#Patient_02.Timepoint_1_unmod',
 '_dyn_#Patient_02.Timepoint_2_unmod',
 '_dyn_#Patient_02.Timepoint_3_unmod',
 '_dyn_#Patient_02.Timepoint_4_unmod',
 '_dyn_#Patient_02.Timepoint_5_unmod',
 '_dyn_#Patient_02.Timepoint_6_unmod',
 '_dyn_#Patient_02.Timepoint_7_unmod',
 '_dyn_#Patient_03.Timepoint_1_unmod',
 '_dyn_#Patient_03.Timepoint_2_unmod',
 '_dyn_#Patient_03.Timepoint_3_unmod',
 '_dyn_#Patient_03.Timepoint_4_unmod',
 '_dyn_#Patient_03.Timepoint_5_unmod',
 '_dyn_#Patient_03.Timepoint_6_unmod',
 '_dyn_#Patient_03.Timepoint_7_unmod',
 '_dyn_#Patient_05.Timepoint_1_unmod',
 '_dyn_#Patient_05.Timepoint_2_unmod',
 '_dyn_#Patient_05.Timepoint_3_unmod',
 '_dyn_

In [240]:
df.drop(intensity_cols, axis=1, inplace=True)

In [241]:
print(df.columns)

Index(['rowid', 'ccms_row_id', 'Peptidoform', 'Peptidoform ID',
       'Unmod peptidoform', 'Total', 'Total- Unmodified sequence',
       'Peptidoforms- Unmodified sequence', 'Proteins', 'Mass',
       ...
       '_dyn_#Patient_M1_healthyMale.Timepoint_1',
       '_dyn_#Patient_M1_healthyMale.Timepoint_2',
       '_dyn_#Patient_M1_healthyMale.Timepoint_3',
       '_dyn_#Patient_M2_healthyMale.Timepoint_1',
       '_dyn_#Patient_M2_healthyMale.Timepoint_2',
       '_dyn_#Patient_M2_healthyMale.Timepoint_3',
       '_dyn_#Patient_M3_healthyMale.Timepoint_1',
       '_dyn_#Patient_M3_healthyMale.Timepoint_2',
       '_dyn_#Patient_M3_healthyMale.Timepoint_3', 'id'],
      dtype='object', length=369)


In [242]:
df.drop(['rowid', 'ccms_row_id', 'Peptidoform ID', 'Mass'],  axis=1, inplace=True)

In [243]:
df.columns

Index(['Peptidoform', 'Unmod peptidoform', 'Total',
       'Total- Unmodified sequence', 'Peptidoforms- Unmodified sequence',
       'Proteins', 'Charge', 'Num Mods', 'All Mods', 'Is Decoy',
       ...
       '_dyn_#Patient_M1_healthyMale.Timepoint_1',
       '_dyn_#Patient_M1_healthyMale.Timepoint_2',
       '_dyn_#Patient_M1_healthyMale.Timepoint_3',
       '_dyn_#Patient_M2_healthyMale.Timepoint_1',
       '_dyn_#Patient_M2_healthyMale.Timepoint_2',
       '_dyn_#Patient_M2_healthyMale.Timepoint_3',
       '_dyn_#Patient_M3_healthyMale.Timepoint_1',
       '_dyn_#Patient_M3_healthyMale.Timepoint_2',
       '_dyn_#Patient_M3_healthyMale.Timepoint_3', 'id'],
      dtype='object', length=365)

In [244]:
df.drop(['Unmod peptidoform', 'Unmod peptidoform', 'Total- Unmodified sequence', 'Peptidoforms- Unmodified sequence', 'Charge', 'Num Mods', 'All Mods', 'Is Decoy'],  axis=1, inplace=True)

In [245]:
for i in df.columns:
    print(i)

Peptidoform
Total
Proteins
Lorikeet input
Orig cluster FDR
Pep Prefix
Annotation
Annotation without position
Known
Num mod frags
PValue
% Explained
Rep cluster task
Rep cluster user
Rep cluster index
Num tasks
Rep spectrum filename
Rep spectrum scan
Outlier groups
Outlier group ratio
Unmod_Peptidoform
_dyn_#Patient_01.Timepoint_1
_dyn_#Patient_01.Timepoint_2
_dyn_#Patient_01.Timepoint_3
_dyn_#Patient_01.Timepoint_4
_dyn_#Patient_01.Timepoint_5
_dyn_#Patient_01.Timepoint_6
_dyn_#Patient_01.Timepoint_7
_dyn_#Patient_02.Timepoint_1
_dyn_#Patient_02.Timepoint_2
_dyn_#Patient_02.Timepoint_3
_dyn_#Patient_02.Timepoint_4
_dyn_#Patient_02.Timepoint_5
_dyn_#Patient_02.Timepoint_6
_dyn_#Patient_02.Timepoint_7
_dyn_#Patient_03.Timepoint_1
_dyn_#Patient_03.Timepoint_2
_dyn_#Patient_03.Timepoint_3
_dyn_#Patient_03.Timepoint_4
_dyn_#Patient_03.Timepoint_5
_dyn_#Patient_03.Timepoint_6
_dyn_#Patient_03.Timepoint_7
_dyn_#Patient_05.Timepoint_1
_dyn_#Patient_05.Timepoint_2
_dyn_#Patient_05.Timepoint_3
_

In [246]:
df.drop(['Total', 'Lorikeet input', 'Orig cluster FDR', 'Pep Prefix',
         'Annotation', 'Annotation without position', 'Known', 
         'Num mod frags', 'PValue',
'Num tasks'],  axis=1, inplace=True)

In [247]:
df.drop([
    "% Explained",
"Rep cluster task",
"Rep cluster user",
"Rep cluster index",
"Rep spectrum filename",
"Rep spectrum scan",
"Outlier groups",
"Outlier group ratio",
"Unmod_Peptidoform"
],  axis=1, inplace=True)

In [248]:
df.head()

,Peptidoform,Proteins,_dyn_#Patient_01.Timepoint_1,_dyn_#Patient_01.Timepoint_2,_dyn_#Patient_01.Timepoint_3,_dyn_#Patient_01.Timepoint_4,_dyn_#Patient_01.Timepoint_5,_dyn_#Patient_01.Timepoint_6,_dyn_#Patient_01.Timepoint_7,_dyn_#Patient_02.Timepoint_1,...,_dyn_#Patient_M1_healthyMale.Timepoint_1,_dyn_#Patient_M1_healthyMale.Timepoint_2,_dyn_#Patient_M1_healthyMale.Timepoint_3,_dyn_#Patient_M2_healthyMale.Timepoint_1,_dyn_#Patient_M2_healthyMale.Timepoint_2,_dyn_#Patient_M2_healthyMale.Timepoint_3,_dyn_#Patient_M3_healthyMale.Timepoint_1,_dyn_#Patient_M3_healthyMale.Timepoint_2,_dyn_#Patient_M3_healthyMale.Timepoint_3,id
0,.SPLFM+15.995GK.,sp|P01009|A1AT_HUMAN;tr|A0A024R6I7|A0A024R6I7_...,"2,173,877,500","2,197,450,000","2,864,495,000","1,613,750,000","2,574,530,000","4,678,075,000","3,746,350,000","1,429,182,500",...,NaN,"21,822,000","25,470,000",NaN,NaN,NaN,NaN,NaN,NaN,0
1,.EPQVYTLPPSREEM+15.995TK.,sp|P01859|IGHG2_HUMAN;sp|P01860|IGHG3_HUMAN;tr...,"2,324,375,000","2,086,445,000","1,523,392,500","930,347,500","1,654,390,000","2,138,075,000","2,543,175,000","1,363,535,000",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,.AVM+15.995DDFAAFVEK.,sp|P02768-2|ALBU_HUMAN;sp|P02768-3|ALBU_HUMAN;...,"7,971,150,000","5,333,875,000","5,175,725,000","4,358,510,000","7,262,825,000","9,100,750,000","7,505,575,000","5,677,100,000",...,NaN,"68,957,000","188,300,000",NaN,NaN,NaN,NaN,NaN,NaN,2
3,.EFNAETFTFHADIC-33.988TLSEK.,sp|P02768-2|ALBU_HUMAN;sp|P02768-3|ALBU_HUMAN;...,"6,637,975,000","3,353,400,000","3,819,700,000","3,478,500,000","4,518,775,000","6,012,960,000","5,263,800,000","3,571,750,000",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
4,.M+15.995ADEAGSEADHEGTHSTK.,sp|P02671-2|FIBA_HUMAN;sp|P02671|FIBA_HUMAN,"2,310,100,000","2,704,025,000","1,235,965,000","1,761,027,500","958,562,500","1,503,225,000","5,039,025,000","1,866,490,000",...,"287,560,000","617,050","339,430,000",NaN,NaN,NaN,NaN,NaN,NaN,4


In [249]:
pattern = r'healthy'
patient_timepoints = []
intensity_cols = [col for col in df.columns if re.search(pattern, col)]
print(intensity_cols)
df.drop(intensity_cols, axis=1, inplace=True)

['_dyn_#Patient_F1_healthyFemale.Timepoint_1', '_dyn_#Patient_F1_healthyFemale.Timepoint_2', '_dyn_#Patient_F1_healthyFemale.Timepoint_3', '_dyn_#Patient_F2_healthyFemale.Timepoint_1', '_dyn_#Patient_F2_healthyFemale.Timepoint_2', '_dyn_#Patient_F2_healthyFemale.Timepoint_3', '_dyn_#Patient_F3_healthyFemale.Timepoint_1', '_dyn_#Patient_F3_healthyFemale.Timepoint_2', '_dyn_#Patient_F3_healthyFemale.Timepoint_3', '_dyn_#Patient_M1_healthyMale.Timepoint_1', '_dyn_#Patient_M1_healthyMale.Timepoint_2', '_dyn_#Patient_M1_healthyMale.Timepoint_3', '_dyn_#Patient_M2_healthyMale.Timepoint_1', '_dyn_#Patient_M2_healthyMale.Timepoint_2', '_dyn_#Patient_M2_healthyMale.Timepoint_3', '_dyn_#Patient_M3_healthyMale.Timepoint_1', '_dyn_#Patient_M3_healthyMale.Timepoint_2', '_dyn_#Patient_M3_healthyMale.Timepoint_3']


In [250]:
df.head()

,Peptidoform,Proteins,_dyn_#Patient_01.Timepoint_1,_dyn_#Patient_01.Timepoint_2,_dyn_#Patient_01.Timepoint_3,_dyn_#Patient_01.Timepoint_4,_dyn_#Patient_01.Timepoint_5,_dyn_#Patient_01.Timepoint_6,_dyn_#Patient_01.Timepoint_7,_dyn_#Patient_02.Timepoint_1,...,_dyn_#Patient_57.Timepoint_6,_dyn_#Patient_57.Timepoint_7,_dyn_#Patient_58.Timepoint_1,_dyn_#Patient_58.Timepoint_2,_dyn_#Patient_58.Timepoint_3,_dyn_#Patient_58.Timepoint_4,_dyn_#Patient_58.Timepoint_5,_dyn_#Patient_58.Timepoint_6,_dyn_#Patient_58.Timepoint_7,id
0,.SPLFM+15.995GK.,sp|P01009|A1AT_HUMAN;tr|A0A024R6I7|A0A024R6I7_...,"2,173,877,500","2,197,450,000","2,864,495,000","1,613,750,000","2,574,530,000","4,678,075,000","3,746,350,000","1,429,182,500",...,"3,107,250,000","4,155,225,000","3,817,200,000","6,073,900,000","6,265,175,000","6,710,850,000","6,563,850,000","4,345,225,000","4,934,400,000",0
1,.EPQVYTLPPSREEM+15.995TK.,sp|P01859|IGHG2_HUMAN;sp|P01860|IGHG3_HUMAN;tr...,"2,324,375,000","2,086,445,000","1,523,392,500","930,347,500","1,654,390,000","2,138,075,000","2,543,175,000","1,363,535,000",...,"1,834,547,500","3,825,300,000","2,355,000,000","2,594,050,000","2,980,550,000","2,852,100,000","2,884,350,000","1,897,400,000","2,802,800,000",1
2,.AVM+15.995DDFAAFVEK.,sp|P02768-2|ALBU_HUMAN;sp|P02768-3|ALBU_HUMAN;...,"7,971,150,000","5,333,875,000","5,175,725,000","4,358,510,000","7,262,825,000","9,100,750,000","7,505,575,000","5,677,100,000",...,"4,462,570,000","7,384,700,000","8,110,325,000","10,065,025,000","10,515,325,000","9,124,475,000","9,814,875,000","6,985,700,000","9,618,975,000",2
3,.EFNAETFTFHADIC-33.988TLSEK.,sp|P02768-2|ALBU_HUMAN;sp|P02768-3|ALBU_HUMAN;...,"6,637,975,000","3,353,400,000","3,819,700,000","3,478,500,000","4,518,775,000","6,012,960,000","5,263,800,000","3,571,750,000",...,"3,267,682,500","5,091,600,000","5,787,700,000","8,006,950,000","8,273,600,000","4,958,575,000","6,853,675,000","6,134,675,000","8,260,200,000",3
4,.M+15.995ADEAGSEADHEGTHSTK.,sp|P02671-2|FIBA_HUMAN;sp|P02671|FIBA_HUMAN,"2,310,100,000","2,704,025,000","1,235,965,000","1,761,027,500","958,562,500","1,503,225,000","5,039,025,000","1,866,490,000",...,"674,722,500","1,378,077,500","1,539,647,500","1,638,325,000","2,350,950,000","1,621,445,000","1,828,525,000","1,205,237,500","1,355,452,500",4


In [251]:
grouped = df.groupby('Proteins')


In [252]:
print(len(grouped))

1308


In [253]:
grouped.head()

,Peptidoform,Proteins,_dyn_#Patient_01.Timepoint_1,_dyn_#Patient_01.Timepoint_2,_dyn_#Patient_01.Timepoint_3,_dyn_#Patient_01.Timepoint_4,_dyn_#Patient_01.Timepoint_5,_dyn_#Patient_01.Timepoint_6,_dyn_#Patient_01.Timepoint_7,_dyn_#Patient_02.Timepoint_1,...,_dyn_#Patient_57.Timepoint_6,_dyn_#Patient_57.Timepoint_7,_dyn_#Patient_58.Timepoint_1,_dyn_#Patient_58.Timepoint_2,_dyn_#Patient_58.Timepoint_3,_dyn_#Patient_58.Timepoint_4,_dyn_#Patient_58.Timepoint_5,_dyn_#Patient_58.Timepoint_6,_dyn_#Patient_58.Timepoint_7,id
0,.SPLFM+15.995GK.,sp|P01009|A1AT_HUMAN;tr|A0A024R6I7|A0A024R6I7_...,"2,173,877,500","2,197,450,000","2,864,495,000","1,613,750,000","2,574,530,000","4,678,075,000","3,746,350,000","1,429,182,500",...,"3,107,250,000","4,155,225,000","3,817,200,000","6,073,900,000","6,265,175,000","6,710,850,000","6,563,850,000","4,345,225,000","4,934,400,000",0
1,.EPQVYTLPPSREEM+15.995TK.,sp|P01859|IGHG2_HUMAN;sp|P01860|IGHG3_HUMAN;tr...,"2,324,375,000","2,086,445,000","1,523,392,500","930,347,500","1,654,390,000","2,138,075,000","2,543,175,000","1,363,535,000",...,"1,834,547,500","3,825,300,000","2,355,000,000","2,594,050,000","2,980,550,000","2,852,100,000","2,884,350,000","1,897,400,000","2,802,800,000",1
2,.AVM+15.995DDFAAFVEK.,sp|P02768-2|ALBU_HUMAN;sp|P02768-3|ALBU_HUMAN;...,"7,971,150,000","5,333,875,000","5,175,725,000","4,358,510,000","7,262,825,000","9,100,750,000","7,505,575,000","5,677,100,000",...,"4,462,570,000","7,384,700,000","8,110,325,000","10,065,025,000","10,515,325,000","9,124,475,000","9,814,875,000","6,985,700,000","9,618,975,000",2
3,.EFNAETFTFHADIC-33.988TLSEK.,sp|P02768-2|ALBU_HUMAN;sp|P02768-3|ALBU_HUMAN;...,"6,637,975,000","3,353,400,000","3,819,700,000","3,478,500,000","4,518,775,000","6,012,960,000","5,263,800,000","3,571,750,000",...,"3,267,682,500","5,091,600,000","5,787,700,000","8,006,950,000","8,273,600,000","4,958,575,000","6,853,675,000","6,134,675,000","8,260,200,000",3
4,.M+15.995ADEAGSEADHEGTHSTK.,sp|P02671-2|FIBA_HUMAN;sp|P02671|FIBA_HUMAN,"2,310,100,000","2,704,025,000","1,235,965,000","1,761,027,500","958,562,500","1,503,225,000","5,039,025,000","1,866,490,000",...,"674,722,500","1,378,077,500","1,539,647,500","1,638,325,000","2,350,950,000","1,621,445,000","1,828,525,000","1,205,237,500","1,355,452,500",4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40836,.-11.98NMFWYQQK.,tr|A0A075B6I3|A0A075B6I3_HUMAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40836
40841,.+119.037RLEEIMK.,sp|Q14244-2|MAP7_HUMAN;sp|Q14244-3|MAP7_HUMAN;...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40841
40855,.+113.91DGAMFFHWRRAAEEGK.,sp|Q9NPF5|DMAP1_HUMAN;tr|Q5TG36|Q5TG36_HUMAN;t...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40855
40897,.-1.01HHHHHHHH.,sp|O00257|CBX4_HUMAN;sp|O00555-2|CAC1A_HUMAN;s...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40897


In [254]:
grouped.get_group('sp|P01834|IGKC_HUMAN').head()


,Peptidoform,Proteins,_dyn_#Patient_01.Timepoint_1,_dyn_#Patient_01.Timepoint_2,_dyn_#Patient_01.Timepoint_3,_dyn_#Patient_01.Timepoint_4,_dyn_#Patient_01.Timepoint_5,_dyn_#Patient_01.Timepoint_6,_dyn_#Patient_01.Timepoint_7,_dyn_#Patient_02.Timepoint_1,...,_dyn_#Patient_57.Timepoint_6,_dyn_#Patient_57.Timepoint_7,_dyn_#Patient_58.Timepoint_1,_dyn_#Patient_58.Timepoint_2,_dyn_#Patient_58.Timepoint_3,_dyn_#Patient_58.Timepoint_4,_dyn_#Patient_58.Timepoint_5,_dyn_#Patient_58.Timepoint_6,_dyn_#Patient_58.Timepoint_7,id
16,.VYAC+305.068EVTHQGLSSPVTK.,sp|P01834|IGKC_HUMAN,"2,289,050,000","1,791,350,000","1,438,220,000","796,622,500","1,897,062,500","1,136,975,000","1,877,952,500","2,878,650,000",...,"2,887,370,000","4,133,000,000","3,874,425,000","4,114,350,000","4,016,050,000","3,517,400,000","4,720,800,000","2,276,950,000","4,766,225,000",16
41,.VYAC-33.988EVTHQGLSSPVTK.,sp|P01834|IGKC_HUMAN,"2,174,875,000","1,385,347,500","1,348,430,000","748,765,000","1,715,380,000","2,071,575,000","1,642,750,000","2,555,525,000",...,"2,788,550,000","3,932,850,000","2,735,075,000","4,045,800,000","4,886,500,000","3,119,450,000","4,815,350,000","3,613,475,000","4,590,950,000",41
46,.VYAC+247.10EVTHQGLSSPVTK.,sp|P01834|IGKC_HUMAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46
73,.HKVYAC-33.988EVTHQGLSSPVTK.,sp|P01834|IGKC_HUMAN,"114,430,250","60,926,250","55,620,750","58,679,750","132,802,000","165,390,000","76,680,750","182,036,500",...,"186,904,250","416,370,000","184,670,000","606,580,000","811,180,000","155,809,000","520,267,500","317,365,000","422,680,000",73
94,.VDNALQSGNSQE+21.982SVTEQDSK.,sp|P01834|IGKC_HUMAN,"255,890,500","212,915,250","64,215,000","162,854,250","127,326,000","268,752,500","258,960,000","281,722,500",...,"391,183,333","420,325,000","481,067,500","590,280,000","602,140,000","516,605,000","528,785,000","428,280,000","527,342,500",94


In [255]:
grouped["Peptidoform"]

In [256]:
# signif_peptidoformes = [
#     ".EI-14.016VLTQSPGTLSLSPGER.",
#     ".AKLPD+14.016ATPTELAK.",
#     ".LLIYDAS+60.036NR.",
#     ".YVLTQPPSVSVAPGQTAR-28.006.",
#     ".+57.021GPSVFPLAPSSK.",
#     ".EVQLVQSGA+26.016EVK.",
#     ".SGQAPVLVIYEDS+14.016K.",
#     ".RPC+57.021FSA+30.011LEVDETYVPK.",
#     ".YVTSAPM+15.995PEPQAPGR.",
#     ".+43.006C+58.005TESLVNR."
# ]

signif_peptidoformes = [
    ".EI-14.016VLTQSPGTLSLSPGER.",
    ".AKLPD+14.016ATPTELAK.",
    ".YVTSAPM+15.995PEPQAPGR.",
    ".LLIYDAS+60.036NR.",
    ".YVLTQPPSVSVAPGQTAR-28.006.",
    ".+57.021GPSVFPLAPSSK.",
    ".EVQLVQSGA+26.016EVK.",
    ".SGQAPVLVIYEDS+14.016K.",
    ".RPC+57.021FSA+30.011LEVDETYVPK.",
    ".+43.006C+58.005TESLVNR."
]

In [257]:
count = 0
all_significant_proteins = []
for group_name, group_data in grouped["Peptidoform"]:
    # Find which values in the group are in signif_peptidoformes
    matching_values = group_data[group_data.isin(signif_peptidoformes)]
    
    if not matching_values.empty:
        count += 1
        print(f"Group Protein: {group_name}")
        print("Matching Peptidoform(s): ", matching_values)
        print("----------")
        all_significant_proteins.append(group_name)

    # break


Group Protein: sp|A0A075B6K4|LV310_HUMAN
Matching Peptidoform(s):  3052    .SGQAPVLVIYEDS+14.016K.
Name: Peptidoform, dtype: object
----------
Group Protein: sp|A0A0A0MRZ8|KVD11_HUMAN;sp|P04433|KV311_HUMAN
Matching Peptidoform(s):  2348    .LLIYDAS+60.036NR.
Name: Peptidoform, dtype: object
----------
Group Protein: sp|A0A0C4DH38|HV551_HUMAN;sp|A0A0G2JMI3|HV692_HUMAN;sp|A0A0J9YXX1|HV5X1_HUMAN
Matching Peptidoform(s):  1704    .EVQLVQSGA+26.016EVK.
Name: Peptidoform, dtype: object
----------
Group Protein: sp|P01619|KV301_HUMAN
Matching Peptidoform(s):  116    .EI-14.016VLTQSPGTLSLSPGER.
Name: Peptidoform, dtype: object
----------
Group Protein: sp|P01857|IGHG1_HUMAN;tr|A0A0A0MS07|A0A0A0MS07_HUMAN;tr|A0A0A0MS08|A0A0A0MS08_HUMAN
Matching Peptidoform(s):  4748    .+57.021GPSVFPLAPSSK.
Name: Peptidoform, dtype: object
----------
Group Protein: sp|P01871-2|IGHM_HUMAN;sp|P01871|IGHM_HUMAN;tr|A0A1B0GUU9|A0A1B0GUU9_HUMAN
Matching Peptidoform(s):  23    .YVTSAPM+15.995PEPQAPGR.
Name: Peptidofor

In [258]:
all_significant_proteins 

['sp|A0A075B6K4|LV310_HUMAN',
 'sp|A0A0A0MRZ8|KVD11_HUMAN;sp|P04433|KV311_HUMAN',
 'sp|A0A0C4DH38|HV551_HUMAN;sp|A0A0G2JMI3|HV692_HUMAN;sp|A0A0J9YXX1|HV5X1_HUMAN',
 'sp|P01619|KV301_HUMAN',
 'sp|P01857|IGHG1_HUMAN;tr|A0A0A0MS07|A0A0A0MS07_HUMAN;tr|A0A0A0MS08|A0A0A0MS08_HUMAN',
 'sp|P01871-2|IGHM_HUMAN;sp|P01871|IGHM_HUMAN;tr|A0A1B0GUU9|A0A1B0GUU9_HUMAN',
 'sp|P02768-2|ALBU_HUMAN;sp|P02768-3|ALBU_HUMAN;sp|P02768|ALBU_HUMAN;tr|A0A087WWT3|A0A087WWT3_HUMAN;tr|A0A0C4DGB6|A0A0C4DGB6_HUMAN;tr|B7WNR0|B7WNR0_HUMAN;tr|C9JKR2|C9JKR2_HUMAN;tr|D6RHD5|D6RHD5_HUMAN;tr|H0YA55|H0YA55_HUMAN',
 'sp|P02774-2|VTDB_HUMAN;sp|P02774-3|VTDB_HUMAN;sp|P02774|VTDB_HUMAN;tr|D6RF35|D6RF35_HUMAN',
 'sp|P80748|LV302_HUMAN']

In [259]:
flat_list = [entry for group in all_significant_proteins for entry in group.split(';')]
flat_list

['sp|A0A075B6K4|LV310_HUMAN',
 'sp|A0A0A0MRZ8|KVD11_HUMAN',
 'sp|P04433|KV311_HUMAN',
 'sp|A0A0C4DH38|HV551_HUMAN',
 'sp|A0A0G2JMI3|HV692_HUMAN',
 'sp|A0A0J9YXX1|HV5X1_HUMAN',
 'sp|P01619|KV301_HUMAN',
 'sp|P01857|IGHG1_HUMAN',
 'tr|A0A0A0MS07|A0A0A0MS07_HUMAN',
 'tr|A0A0A0MS08|A0A0A0MS08_HUMAN',
 'sp|P01871-2|IGHM_HUMAN',
 'sp|P01871|IGHM_HUMAN',
 'tr|A0A1B0GUU9|A0A1B0GUU9_HUMAN',
 'sp|P02768-2|ALBU_HUMAN',
 'sp|P02768-3|ALBU_HUMAN',
 'sp|P02768|ALBU_HUMAN',
 'tr|A0A087WWT3|A0A087WWT3_HUMAN',
 'tr|A0A0C4DGB6|A0A0C4DGB6_HUMAN',
 'tr|B7WNR0|B7WNR0_HUMAN',
 'tr|C9JKR2|C9JKR2_HUMAN',
 'tr|D6RHD5|D6RHD5_HUMAN',
 'tr|H0YA55|H0YA55_HUMAN',
 'sp|P02774-2|VTDB_HUMAN',
 'sp|P02774-3|VTDB_HUMAN',
 'sp|P02774|VTDB_HUMAN',
 'tr|D6RF35|D6RF35_HUMAN',
 'sp|P80748|LV302_HUMAN']

In [260]:
seen = set()
unique_list = []
for item in flat_list:
    if item not in seen:
        unique_list.append(item)
        seen.add(item)

unique_list, len(unique_list)

(['sp|A0A075B6K4|LV310_HUMAN',
  'sp|A0A0A0MRZ8|KVD11_HUMAN',
  'sp|P04433|KV311_HUMAN',
  'sp|A0A0C4DH38|HV551_HUMAN',
  'sp|A0A0G2JMI3|HV692_HUMAN',
  'sp|A0A0J9YXX1|HV5X1_HUMAN',
  'sp|P01619|KV301_HUMAN',
  'sp|P01857|IGHG1_HUMAN',
  'tr|A0A0A0MS07|A0A0A0MS07_HUMAN',
  'tr|A0A0A0MS08|A0A0A0MS08_HUMAN',
  'sp|P01871-2|IGHM_HUMAN',
  'sp|P01871|IGHM_HUMAN',
  'tr|A0A1B0GUU9|A0A1B0GUU9_HUMAN',
  'sp|P02768-2|ALBU_HUMAN',
  'sp|P02768-3|ALBU_HUMAN',
  'sp|P02768|ALBU_HUMAN',
  'tr|A0A087WWT3|A0A087WWT3_HUMAN',
  'tr|A0A0C4DGB6|A0A0C4DGB6_HUMAN',
  'tr|B7WNR0|B7WNR0_HUMAN',
  'tr|C9JKR2|C9JKR2_HUMAN',
  'tr|D6RHD5|D6RHD5_HUMAN',
  'tr|H0YA55|H0YA55_HUMAN',
  'sp|P02774-2|VTDB_HUMAN',
  'sp|P02774-3|VTDB_HUMAN',
  'sp|P02774|VTDB_HUMAN',
  'tr|D6RF35|D6RF35_HUMAN',
  'sp|P80748|LV302_HUMAN'],
 27)

In [261]:
all_significant_proteins = ['sp|P01619|KV301_HUMAN', 'sp|A0A075B6K4|LV310_HUMAN', 'sp|P80748|LV302_HUMAN', 'tr|A0A0A0MS08|A0A0A0MS08_HUMAN',
                            'sp|P01743|HV146_HUMAN', 'sp|A0A0A0MRZ8|KVD11_HUMAN', 'sp|P02774|VTDB_HUMAN', 'tr|D6RHD5|D6RHD5_HUMAN',
                            'sp|P01871|IGHM_HUMAN']
# Parsimony Algorithm

In [262]:
# all_significant_proteins = unique_list

In [263]:
df[df["Proteins"] == 'sp|P01619|KV301_HUMAN'].shape

(177, 321)

In [264]:
df[df["Proteins"] == 'sp|P01871|IGHM_HUMAN'].shape

(0, 321)

In [265]:
df["Proteins"]

0        sp|P01009|A1AT_HUMAN;tr|A0A024R6I7|A0A024R6I7_...
1        sp|P01859|IGHG2_HUMAN;sp|P01860|IGHG3_HUMAN;tr...
2        sp|P02768-2|ALBU_HUMAN;sp|P02768-3|ALBU_HUMAN;...
3        sp|P02768-2|ALBU_HUMAN;sp|P02768-3|ALBU_HUMAN;...
4              sp|P02671-2|FIBA_HUMAN;sp|P02671|FIBA_HUMAN
                               ...                        
40916    sp|P0C0L4-2|CO4A_HUMAN;sp|P0C0L4|CO4A_HUMAN;sp...
40917          sp|P02671-2|FIBA_HUMAN;sp|P02671|FIBA_HUMAN
40918    sp|P02768-3|ALBU_HUMAN;sp|P02768|ALBU_HUMAN;tr...
40919                                 sp|P01834|IGKC_HUMAN
40920    sp|P02768-2|ALBU_HUMAN;sp|P02768-3|ALBU_HUMAN;...
Name: Proteins, Length: 40921, dtype: object

In [266]:
for i in range(len(all_significant_proteins)):
    filtered_df = df[all_significant_proteins[i] == df["Proteins"]]
    # file_name = f"{all_significant_proteins[i]}.csv"
    file_path = os.path.join(f"./data/filetered_proteins/{i+1}.csv")
    filtered_df.to_csv(file_path, index=True)
    print(f"Saved: {file_path}")
    print(filtered_df.shape)

Saved: ./data/filetered_proteins/1.csv
(177, 321)
Saved: ./data/filetered_proteins/2.csv
(35, 321)
Saved: ./data/filetered_proteins/3.csv
(143, 321)
Saved: ./data/filetered_proteins/4.csv
(0, 321)
Saved: ./data/filetered_proteins/5.csv
(5, 321)
Saved: ./data/filetered_proteins/6.csv
(0, 321)
Saved: ./data/filetered_proteins/7.csv
(0, 321)
Saved: ./data/filetered_proteins/8.csv
(6, 321)
Saved: ./data/filetered_proteins/9.csv
(0, 321)


In [267]:
# for i, protein in enumerate(all_significant_proteins, start=1):
#     filtered_df = df[df["Proteins"].str.contains(protein, na=False)]
    
#     file_path = os.path.join(f"./data/filetered_proteins/{i}.csv")
#     os.makedirs(os.path.dirname(file_path), exist_ok=True)
#     filtered_df.to_csv(file_path, index=True)
    
#     print(filtered_df.shape)



In [268]:
df = pd.read_csv("./data/filetered_proteins/1.csv")
df.head()

,Unnamed: 0,Peptidoform,Proteins,_dyn_#Patient_01.Timepoint_1,_dyn_#Patient_01.Timepoint_2,_dyn_#Patient_01.Timepoint_3,_dyn_#Patient_01.Timepoint_4,_dyn_#Patient_01.Timepoint_5,_dyn_#Patient_01.Timepoint_6,_dyn_#Patient_01.Timepoint_7,...,_dyn_#Patient_57.Timepoint_6,_dyn_#Patient_57.Timepoint_7,_dyn_#Patient_58.Timepoint_1,_dyn_#Patient_58.Timepoint_2,_dyn_#Patient_58.Timepoint_3,_dyn_#Patient_58.Timepoint_4,_dyn_#Patient_58.Timepoint_5,_dyn_#Patient_58.Timepoint_6,_dyn_#Patient_58.Timepoint_7,id
0,116,.EI-14.016VLTQSPGTLSLSPGER.,sp|P01619|KV301_HUMAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"190,134,500","143,996,000","245,581,000","326,225,000","410,935,000","260,945,000","327,880,000","224,129,250","347,017,500",116
1,326,.EIVLTQSPGTLSLSPGE-14.016R.,sp|P01619|KV301_HUMAN,"167,282,000","116,683,250","115,415,250","93,152,250","113,858,500","129,512,500","137,566,000",...,"94,435,500","90,822,250","126,249,750","149,067,750","173,519,500","130,186,250","165,474,250","122,781,250","186,445,500",326
2,374,.-18.011EIVLTQSPGTLSLSPGER.,sp|P01619|KV301_HUMAN,"96,934,000","72,778,000","52,651,250","45,407,667","55,824,250","83,978,333","67,884,750",...,"63,233,250","66,445,000","118,900,750","150,684,750","142,990,000","138,161,750","155,659,000","113,215,000","144,106,500",374
3,757,.EIVLTQSPGTLSL-14.016SPGER.,sp|P01619|KV301_HUMAN,"113,399,250","73,275,000","89,338,750","69,790,250","80,708,500","87,992,750","73,728,667",...,"219,641,250","204,925,250","29,165,333",NaN,"24,706,500","21,912,000","64,389,000","28,313,500","39,624,500",757
4,904,.E-14.016IVLTQSPGTLSLSPGER.,sp|P01619|KV301_HUMAN,"232,722,000","169,428,500","161,700,750","148,550,000","133,551,250","153,596,750","173,652,500",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,904


In [269]:
df.head(), df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177 entries, 0 to 176
Columns: 322 entries, Unnamed: 0 to id
dtypes: int64(2), object(320)
memory usage: 445.4+ KB


(   Unnamed: 0                  Peptidoform               Proteins  \
 0         116  .EI-14.016VLTQSPGTLSLSPGER.  sp|P01619|KV301_HUMAN   
 1         326  .EIVLTQSPGTLSLSPGE-14.016R.  sp|P01619|KV301_HUMAN   
 2         374  .-18.011EIVLTQSPGTLSLSPGER.  sp|P01619|KV301_HUMAN   
 3         757  .EIVLTQSPGTLSL-14.016SPGER.  sp|P01619|KV301_HUMAN   
 4         904  .E-14.016IVLTQSPGTLSLSPGER.  sp|P01619|KV301_HUMAN   
 
   _dyn_#Patient_01.Timepoint_1 _dyn_#Patient_01.Timepoint_2  \
 0                          NaN                          NaN   
 1                  167,282,000                  116,683,250   
 2                   96,934,000                   72,778,000   
 3                  113,399,250                   73,275,000   
 4                  232,722,000                  169,428,500   
 
   _dyn_#Patient_01.Timepoint_3 _dyn_#Patient_01.Timepoint_4  \
 0                          NaN                          NaN   
 1                  115,415,250                   93,152,250   


In [270]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [271]:
df.head()

,Peptidoform,Proteins,_dyn_#Patient_01.Timepoint_1,_dyn_#Patient_01.Timepoint_2,_dyn_#Patient_01.Timepoint_3,_dyn_#Patient_01.Timepoint_4,_dyn_#Patient_01.Timepoint_5,_dyn_#Patient_01.Timepoint_6,_dyn_#Patient_01.Timepoint_7,_dyn_#Patient_02.Timepoint_1,...,_dyn_#Patient_57.Timepoint_6,_dyn_#Patient_57.Timepoint_7,_dyn_#Patient_58.Timepoint_1,_dyn_#Patient_58.Timepoint_2,_dyn_#Patient_58.Timepoint_3,_dyn_#Patient_58.Timepoint_4,_dyn_#Patient_58.Timepoint_5,_dyn_#Patient_58.Timepoint_6,_dyn_#Patient_58.Timepoint_7,id
0,.EI-14.016VLTQSPGTLSLSPGER.,sp|P01619|KV301_HUMAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"311,145,000",...,"190,134,500","143,996,000","245,581,000","326,225,000","410,935,000","260,945,000","327,880,000","224,129,250","347,017,500",116
1,.EIVLTQSPGTLSLSPGE-14.016R.,sp|P01619|KV301_HUMAN,"167,282,000","116,683,250","115,415,250","93,152,250","113,858,500","129,512,500","137,566,000","134,213,500",...,"94,435,500","90,822,250","126,249,750","149,067,750","173,519,500","130,186,250","165,474,250","122,781,250","186,445,500",326
2,.-18.011EIVLTQSPGTLSLSPGER.,sp|P01619|KV301_HUMAN,"96,934,000","72,778,000","52,651,250","45,407,667","55,824,250","83,978,333","67,884,750","122,421,000",...,"63,233,250","66,445,000","118,900,750","150,684,750","142,990,000","138,161,750","155,659,000","113,215,000","144,106,500",374
3,.EIVLTQSPGTLSL-14.016SPGER.,sp|P01619|KV301_HUMAN,"113,399,250","73,275,000","89,338,750","69,790,250","80,708,500","87,992,750","73,728,667","124,395,333",...,"219,641,250","204,925,250","29,165,333",NaN,"24,706,500","21,912,000","64,389,000","28,313,500","39,624,500",757
4,.E-14.016IVLTQSPGTLSLSPGER.,sp|P01619|KV301_HUMAN,"232,722,000","169,428,500","161,700,750","148,550,000","133,551,250","153,596,750","173,652,500",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,904


In [272]:
threshold = 0.6  # 90% threshold
max_na = df.shape[1] * threshold

df = df[~((df.isna() | (df == '')).sum(axis=1) > max_na)].reset_index(drop=True)


In [273]:
df.shape

(20, 321)

In [274]:
df.head()

,Peptidoform,Proteins,_dyn_#Patient_01.Timepoint_1,_dyn_#Patient_01.Timepoint_2,_dyn_#Patient_01.Timepoint_3,_dyn_#Patient_01.Timepoint_4,_dyn_#Patient_01.Timepoint_5,_dyn_#Patient_01.Timepoint_6,_dyn_#Patient_01.Timepoint_7,_dyn_#Patient_02.Timepoint_1,...,_dyn_#Patient_57.Timepoint_6,_dyn_#Patient_57.Timepoint_7,_dyn_#Patient_58.Timepoint_1,_dyn_#Patient_58.Timepoint_2,_dyn_#Patient_58.Timepoint_3,_dyn_#Patient_58.Timepoint_4,_dyn_#Patient_58.Timepoint_5,_dyn_#Patient_58.Timepoint_6,_dyn_#Patient_58.Timepoint_7,id
0,.EI-14.016VLTQSPGTLSLSPGER.,sp|P01619|KV301_HUMAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"311,145,000",...,"190,134,500","143,996,000","245,581,000","326,225,000","410,935,000","260,945,000","327,880,000","224,129,250","347,017,500",116
1,.EIVLTQSPGTLSLSPGE-14.016R.,sp|P01619|KV301_HUMAN,"167,282,000","116,683,250","115,415,250","93,152,250","113,858,500","129,512,500","137,566,000","134,213,500",...,"94,435,500","90,822,250","126,249,750","149,067,750","173,519,500","130,186,250","165,474,250","122,781,250","186,445,500",326
2,.-18.011EIVLTQSPGTLSLSPGER.,sp|P01619|KV301_HUMAN,"96,934,000","72,778,000","52,651,250","45,407,667","55,824,250","83,978,333","67,884,750","122,421,000",...,"63,233,250","66,445,000","118,900,750","150,684,750","142,990,000","138,161,750","155,659,000","113,215,000","144,106,500",374
3,.EIVLTQSPGTLSL-14.016SPGER.,sp|P01619|KV301_HUMAN,"113,399,250","73,275,000","89,338,750","69,790,250","80,708,500","87,992,750","73,728,667","124,395,333",...,"219,641,250","204,925,250","29,165,333",NaN,"24,706,500","21,912,000","64,389,000","28,313,500","39,624,500",757
4,.EI-12.036VLTQSPGTLSLSPGER.,sp|P01619|KV301_HUMAN,"186,952,750","117,938,250","84,827,000","114,645,500","57,233,250","91,217,667","123,316,000","111,493,500",...,"132,457,250","120,154,250","91,058,000","95,677,000","109,954,750","74,979,000","116,400,333","82,698,000","133,080,000",1049


In [275]:
object_cols = df.select_dtypes(include='object').columns

# Convert comma-separated numeric strings to actual numbers
for col in object_cols:
    if col not in ['Peptidoform', 'Proteins']:
        df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', ''), errors='coerce')

df.head()


,Peptidoform,Proteins,_dyn_#Patient_01.Timepoint_1,_dyn_#Patient_01.Timepoint_2,_dyn_#Patient_01.Timepoint_3,_dyn_#Patient_01.Timepoint_4,_dyn_#Patient_01.Timepoint_5,_dyn_#Patient_01.Timepoint_6,_dyn_#Patient_01.Timepoint_7,_dyn_#Patient_02.Timepoint_1,...,_dyn_#Patient_57.Timepoint_6,_dyn_#Patient_57.Timepoint_7,_dyn_#Patient_58.Timepoint_1,_dyn_#Patient_58.Timepoint_2,_dyn_#Patient_58.Timepoint_3,_dyn_#Patient_58.Timepoint_4,_dyn_#Patient_58.Timepoint_5,_dyn_#Patient_58.Timepoint_6,_dyn_#Patient_58.Timepoint_7,id
0,.EI-14.016VLTQSPGTLSLSPGER.,sp|P01619|KV301_HUMAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,311145000.0,...,190134500.0,143996000.0,245581000.0,326225000.0,410935000.0,260945000.0,327880000.0,224129250.0,347017500.0,116
1,.EIVLTQSPGTLSLSPGE-14.016R.,sp|P01619|KV301_HUMAN,167282000.0,116683250.0,115415250.0,93152250.0,113858500.0,129512500.0,137566000.0,134213500.0,...,94435500.0,90822250.0,126249750.0,149067750.0,173519500.0,130186250.0,165474250.0,122781250.0,186445500.0,326
2,.-18.011EIVLTQSPGTLSLSPGER.,sp|P01619|KV301_HUMAN,96934000.0,72778000.0,52651250.0,45407667.0,55824250.0,83978333.0,67884750.0,122421000.0,...,63233250.0,66445000.0,118900750.0,150684750.0,142990000.0,138161750.0,155659000.0,113215000.0,144106500.0,374
3,.EIVLTQSPGTLSL-14.016SPGER.,sp|P01619|KV301_HUMAN,113399250.0,73275000.0,89338750.0,69790250.0,80708500.0,87992750.0,73728667.0,124395333.0,...,219641250.0,204925250.0,29165333.0,NaN,24706500.0,21912000.0,64389000.0,28313500.0,39624500.0,757
4,.EI-12.036VLTQSPGTLSLSPGER.,sp|P01619|KV301_HUMAN,186952750.0,117938250.0,84827000.0,114645500.0,57233250.0,91217667.0,123316000.0,111493500.0,...,132457250.0,120154250.0,91058000.0,95677000.0,109954750.0,74979000.0,116400333.0,82698000.0,133080000.0,1049


In [276]:
timepoints = [col for col in df.columns if any(tp in col.lower() for tp in ['_1', '_2', '_3', '_4', '_5', '_6', '_7'])]

for index, row in df.iterrows():
    for col in timepoints:
        if pd.isna(row[col]):
            if df[col].dtype in ['float64', 'int64']:
                median_val = row[timepoints].median(skipna=True)
                df.at[index, col] = median_val
            elif df[col].dtype == 'object':
                mode_val = row[timepoints].mode(dropna=True)
                if not mode_val.empty:
                    df.at[index, col] = mode_val[0]



In [277]:
df.columns

Index(['Peptidoform', 'Proteins', '_dyn_#Patient_01.Timepoint_1',
       '_dyn_#Patient_01.Timepoint_2', '_dyn_#Patient_01.Timepoint_3',
       '_dyn_#Patient_01.Timepoint_4', '_dyn_#Patient_01.Timepoint_5',
       '_dyn_#Patient_01.Timepoint_6', '_dyn_#Patient_01.Timepoint_7',
       '_dyn_#Patient_02.Timepoint_1',
       ...
       '_dyn_#Patient_57.Timepoint_6', '_dyn_#Patient_57.Timepoint_7',
       '_dyn_#Patient_58.Timepoint_1', '_dyn_#Patient_58.Timepoint_2',
       '_dyn_#Patient_58.Timepoint_3', '_dyn_#Patient_58.Timepoint_4',
       '_dyn_#Patient_58.Timepoint_5', '_dyn_#Patient_58.Timepoint_6',
       '_dyn_#Patient_58.Timepoint_7', 'id'],
      dtype='object', length=321)

In [278]:
df.head()

,Peptidoform,Proteins,_dyn_#Patient_01.Timepoint_1,_dyn_#Patient_01.Timepoint_2,_dyn_#Patient_01.Timepoint_3,_dyn_#Patient_01.Timepoint_4,_dyn_#Patient_01.Timepoint_5,_dyn_#Patient_01.Timepoint_6,_dyn_#Patient_01.Timepoint_7,_dyn_#Patient_02.Timepoint_1,...,_dyn_#Patient_57.Timepoint_6,_dyn_#Patient_57.Timepoint_7,_dyn_#Patient_58.Timepoint_1,_dyn_#Patient_58.Timepoint_2,_dyn_#Patient_58.Timepoint_3,_dyn_#Patient_58.Timepoint_4,_dyn_#Patient_58.Timepoint_5,_dyn_#Patient_58.Timepoint_6,_dyn_#Patient_58.Timepoint_7,id
0,.EI-14.016VLTQSPGTLSLSPGER.,sp|P01619|KV301_HUMAN,306965000.0,306965000.0,306965000.0,306965000.0,306965000.0,306965000.0,306965000.0,311145000.0,...,190134500.0,143996000.0,245581000.0,326225000.0,410935000.0,260945000.0,327880000.0,224129250.0,347017500.0,116
1,.EIVLTQSPGTLSLSPGE-14.016R.,sp|P01619|KV301_HUMAN,167282000.0,116683250.0,115415250.0,93152250.0,113858500.0,129512500.0,137566000.0,134213500.0,...,94435500.0,90822250.0,126249750.0,149067750.0,173519500.0,130186250.0,165474250.0,122781250.0,186445500.0,326
2,.-18.011EIVLTQSPGTLSLSPGER.,sp|P01619|KV301_HUMAN,96934000.0,72778000.0,52651250.0,45407667.0,55824250.0,83978333.0,67884750.0,122421000.0,...,63233250.0,66445000.0,118900750.0,150684750.0,142990000.0,138161750.0,155659000.0,113215000.0,144106500.0,374
3,.EIVLTQSPGTLSL-14.016SPGER.,sp|P01619|KV301_HUMAN,113399250.0,73275000.0,89338750.0,69790250.0,80708500.0,87992750.0,73728667.0,124395333.0,...,219641250.0,204925250.0,29165333.0,64389000.0,24706500.0,21912000.0,64389000.0,28313500.0,39624500.0,757
4,.EI-12.036VLTQSPGTLSLSPGER.,sp|P01619|KV301_HUMAN,186952750.0,117938250.0,84827000.0,114645500.0,57233250.0,91217667.0,123316000.0,111493500.0,...,132457250.0,120154250.0,91058000.0,95677000.0,109954750.0,74979000.0,116400333.0,82698000.0,133080000.0,1049


In [279]:
patient_cols = [col for col in df.columns if col.startswith('_dyn_#Patient_') and '.Timepoint_' in col]
df[patient_cols] = np.log2(df[patient_cols] + 1)

In [280]:
df.head()

,Peptidoform,Proteins,_dyn_#Patient_01.Timepoint_1,_dyn_#Patient_01.Timepoint_2,_dyn_#Patient_01.Timepoint_3,_dyn_#Patient_01.Timepoint_4,_dyn_#Patient_01.Timepoint_5,_dyn_#Patient_01.Timepoint_6,_dyn_#Patient_01.Timepoint_7,_dyn_#Patient_02.Timepoint_1,...,_dyn_#Patient_57.Timepoint_6,_dyn_#Patient_57.Timepoint_7,_dyn_#Patient_58.Timepoint_1,_dyn_#Patient_58.Timepoint_2,_dyn_#Patient_58.Timepoint_3,_dyn_#Patient_58.Timepoint_4,_dyn_#Patient_58.Timepoint_5,_dyn_#Patient_58.Timepoint_6,_dyn_#Patient_58.Timepoint_7,id
0,.EI-14.016VLTQSPGTLSLSPGER.,sp|P01619|KV301_HUMAN,28.193499,28.193499,28.193499,28.193499,28.193499,28.193499,28.193499,28.213012,...,27.502445,27.101454,27.871624,28.281292,28.614335,27.959171,28.288593,27.739756,28.370433,116
1,.EIVLTQSPGTLSLSPGE-14.016R.,sp|P01619|KV301_HUMAN,27.317707,26.798022,26.782259,26.473087,26.762667,26.948516,27.035549,26.999955,...,26.492826,26.436542,26.911705,27.151393,27.370523,26.956002,27.302031,26.871515,27.474179,326
2,.-18.011EIVLTQSPGTLSLSPGER.,sp|P01619|KV301_HUMAN,26.530499,26.116999,25.649964,25.436433,25.734389,26.323514,26.016584,26.867276,...,25.914180,25.985657,26.825183,27.166958,27.091339,27.041783,27.213814,26.754490,27.102560,374
3,.EIVLTQSPGTLSL-14.016SPGER.,sp|P01619|KV301_HUMAN,26.756836,26.126818,26.412783,26.056522,26.266217,26.390881,26.135722,26.890357,...,27.710574,27.610523,24.797751,25.940311,24.558387,24.385218,25.940311,24.754987,25.239889,757
4,.EI-12.036VLTQSPGTLSLSPGER.,sp|P01619|KV301_HUMAN,27.478098,26.813456,26.338020,26.772604,25.770350,26.442810,26.877785,26.732384,...,26.980952,26.840312,26.440282,26.511669,26.712335,26.159983,26.794520,26.301349,26.987719,1049


In [281]:
df.columns

Index(['Peptidoform', 'Proteins', '_dyn_#Patient_01.Timepoint_1',
       '_dyn_#Patient_01.Timepoint_2', '_dyn_#Patient_01.Timepoint_3',
       '_dyn_#Patient_01.Timepoint_4', '_dyn_#Patient_01.Timepoint_5',
       '_dyn_#Patient_01.Timepoint_6', '_dyn_#Patient_01.Timepoint_7',
       '_dyn_#Patient_02.Timepoint_1',
       ...
       '_dyn_#Patient_57.Timepoint_6', '_dyn_#Patient_57.Timepoint_7',
       '_dyn_#Patient_58.Timepoint_1', '_dyn_#Patient_58.Timepoint_2',
       '_dyn_#Patient_58.Timepoint_3', '_dyn_#Patient_58.Timepoint_4',
       '_dyn_#Patient_58.Timepoint_5', '_dyn_#Patient_58.Timepoint_6',
       '_dyn_#Patient_58.Timepoint_7', 'id'],
      dtype='object', length=321)

In [282]:

patient_cols = [col for col in df.columns if col.startswith('_dyn_#Patient_') and '.Timepoint_' in col]

def normalize_row(row):
    min_val = row.min()
    max_val = row.max()
    if min_val == max_val:
        return pd.Series([0] * len(row), index=row.index)  # or keep it as-is
    return 2 * ((row - min_val) / (max_val - min_val)) - 1

df[patient_cols] = df[patient_cols].apply(normalize_row, axis=1)


In [283]:
df.head()

,Peptidoform,Proteins,_dyn_#Patient_01.Timepoint_1,_dyn_#Patient_01.Timepoint_2,_dyn_#Patient_01.Timepoint_3,_dyn_#Patient_01.Timepoint_4,_dyn_#Patient_01.Timepoint_5,_dyn_#Patient_01.Timepoint_6,_dyn_#Patient_01.Timepoint_7,_dyn_#Patient_02.Timepoint_1,...,_dyn_#Patient_57.Timepoint_6,_dyn_#Patient_57.Timepoint_7,_dyn_#Patient_58.Timepoint_1,_dyn_#Patient_58.Timepoint_2,_dyn_#Patient_58.Timepoint_3,_dyn_#Patient_58.Timepoint_4,_dyn_#Patient_58.Timepoint_5,_dyn_#Patient_58.Timepoint_6,_dyn_#Patient_58.Timepoint_7,id
0,.EI-14.016VLTQSPGTLSLSPGER.,sp|P01619|KV301_HUMAN,-0.021647,-0.021647,-0.021647,-0.021647,-0.021647,-0.021647,-0.021647,-0.007450,...,-0.524419,-0.816158,-0.255825,0.042227,0.284530,-0.192131,0.047538,-0.351765,0.107081,116
1,.EIVLTQSPGTLSLSPGE-14.016R.,sp|P01619|KV301_HUMAN,-0.061012,-0.286705,-0.293551,-0.427821,-0.302060,-0.221347,-0.183550,-0.199008,...,-0.419249,-0.443692,-0.237334,-0.133240,-0.038075,-0.218096,-0.067820,-0.254788,0.006942,326
2,.-18.011EIVLTQSPGTLSLSPGER.,sp|P01619|KV301_HUMAN,0.047810,-0.209020,-0.499100,-0.631728,-0.446664,-0.080751,-0.271389,0.256986,...,-0.334993,-0.290598,0.230841,0.443122,0.396154,0.365374,0.472224,0.186933,0.403124,374
3,.EIVLTQSPGTLSL-14.016SPGER.,sp|P01619|KV301_HUMAN,0.182983,-0.037706,0.062464,-0.062330,0.011124,0.054793,-0.034587,0.229754,...,0.517067,0.482020,-0.503264,-0.103037,-0.587110,-0.647770,-0.103037,-0.518244,-0.348387,757
4,.EI-12.036VLTQSPGTLSLSPGER.,sp|P01619|KV301_HUMAN,0.392732,0.073420,-0.154992,0.053794,-0.427716,-0.104649,0.104325,0.034471,...,0.153889,0.086322,-0.105863,-0.071567,0.024838,-0.240526,0.064322,-0.172610,0.157140,1049


In [284]:

patient_timepoints = defaultdict(list)
for col in patient_cols:
    parts = col.split('.Timepoint_')
    if len(parts) == 2:
        patient_id = parts[0]
        patient_timepoints[patient_id].append(col)

differences = pd.DataFrame()
for patient_id, cols in patient_timepoints.items():
    sorted_cols = sorted(cols, key=lambda x: int(x.split('.Timepoint_')[1]))
    first_col = sorted_cols[0]
    last_col = sorted_cols[-1]
    diff_col_name = f"{patient_id}_range"
    differences[diff_col_name] = df[last_col] - df[first_col]


In [285]:
# differences['Peptidoform'] = df['Peptidoform']
# differences['Proteins'] = df['Proteins']
print(differences.shape)
differences.head()

(20, 52)


,_dyn_#Patient_01_range,_dyn_#Patient_02_range,_dyn_#Patient_03_range,_dyn_#Patient_05_range,_dyn_#Patient_06_range,_dyn_#Patient_07_range,_dyn_#Patient_08_range,_dyn_#Patient_09_range,_dyn_#Patient_10_range,_dyn_#Patient_11_range,...,_dyn_#Patient_47_range,_dyn_#Patient_48_range,_dyn_#Patient_49_range,_dyn_#Patient_50_range,_dyn_#Patient_51_range,_dyn_#Patient_52_range,_dyn_#Patient_53_range,_dyn_#Patient_54_range,_dyn_#Patient_57_range,_dyn_#Patient_58_range
0,0.000000,-0.060019,-0.103273,-0.412123,0.110244,0.141867,-0.037401,0.555304,0.547398,0.403172,...,0.000000,0.307048,0.000000,0.020549,0.436085,-0.396349,-0.050769,0.000000,-0.129743,0.362906
1,-0.122538,0.101052,-0.230063,-0.174578,0.139229,0.387637,-0.051608,-0.046669,0.203060,0.302248,...,-0.225819,0.202217,0.159340,0.090846,0.541369,0.062224,-0.167606,0.310153,-0.161973,0.244276
2,-0.319199,-0.207359,-0.306222,0.173678,-0.218996,0.232151,-0.154688,0.138218,-0.125296,0.318227,...,0.228288,0.770506,-0.208372,0.107900,0.421416,-0.101689,0.140215,0.008298,0.105641,0.172282
3,-0.217569,-0.220957,-0.846837,-0.191044,-0.110528,0.140063,-0.420556,-0.358837,-0.185907,0.486708,...,-0.213080,-0.161348,-0.021753,-0.085841,-0.002906,-0.011220,-0.164948,0.385145,-0.093063,0.154876
4,-0.288407,0.026621,-0.156019,0.051548,0.030106,0.326838,-0.123338,0.305859,0.051564,0.636060,...,-0.097651,0.166934,-0.128518,0.162067,0.000000,0.000000,-0.105913,0.245739,0.147739,0.263003


In [286]:
values = differences.mean(axis=1)
values

0     0.026758
1     0.069578
2     0.081440
3    -0.033348
4     0.053744
5     0.004055
6     0.049501
7     0.054366
8     0.025694
9     0.092197
10    0.072039
11    0.068641
12   -0.046192
13   -0.004596
14    0.120439
15    0.021607
16    0.026024
17   -0.008768
18    0.039541
19    0.047229
dtype: float64

In [287]:
file_path = os.path.join(f"./data/filetered_proteins/{+1}.csv")
filtered_df.to_csv(file_path, index=True)

In [ ]:
rootDir = './data/filetered_proteins/'
for filename in os.listdir(rootDir):
    df = pd.read_csv(rootDir+filename)
    # print(df.head())
    df.drop('Unnamed: 0', axis=1, inplace=True)
    print(f"Data Shape for {filename} is {df.shape}")
    threshold = 0.9  # 90% threshold
    max_na = df.shape[1] * threshold

    df = df[~((df.isna() | (df == '')).sum(axis=1) > max_na)].reset_index(drop=True)
    print(f"Data Shape for {filename} after dropping 90% data {df.shape}")
    object_cols = df.select_dtypes(include='object').columns

    # Convert comma-separated numeric strings to actual numbers
    for col in object_cols:
        if col not in ['Peptidoform', 'Proteins']:
            df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', ''), errors='coerce')


    timepoints = [col for col in df.columns if any(tp in col.lower() for tp in ['_1', '_2', '_3', '_4', '_5', '_6', '_7'])]


    for index, row in df.iterrows():
        for col in timepoints:
            if pd.isna(row[col]):
                if df[col].dtype in ['float64', 'int64']:
                    median_val = row[timepoints].median(skipna=True)
                    df.at[index, col] = median_val
                elif df[col].dtype == 'object':
                    mode_val = row[timepoints].mode(dropna=True)
                    if not mode_val.empty:
                        df.at[index, col] = mode_val[0]

    patient_cols = [col for col in df.columns if col.startswith('_dyn_#Patient_') and '.Timepoint_' in col]
    df[patient_cols] = np.log2(df[patient_cols] + 1)
    df[patient_cols] = df[patient_cols].apply(normalize_row, axis=1)


    patient_timepoints = defaultdict(list)
    for col in patient_cols:
        parts = col.split('.Timepoint_')
        if len(parts) == 2:
            patient_id = parts[0]
            patient_timepoints[patient_id].append(col)

    initial_timeFrames = []
    final_timeFrames = []
    differences = pd.DataFrame()
    for patient_id, cols in patient_timepoints.items():
        sorted_cols = sorted(cols, key=lambda x: int(x.split('.Timepoint_')[1]))
        first_col = sorted_cols[0]
        last_col = sorted_cols[-1]
        diff_col_name = f"{patient_id}_range"
        change = df[last_col] - df[first_col]
        differences[diff_col_name] = (change/df[first_col]) * 100
        initial_timeFrames.append(df[first_col])
        final_timeFrames.append(df[last_col])
    
    print(len(initial_timeFrames), len(final_timeFrames))
    Tfirst_values = np.array(initial_timeFrames, dtype = float)  # Columns 1 to 58 (P1_T1 to P58_T1)
    Tlast_values = np.array(final_timeFrames, dtype = float) # Columns 59 to 116 (P1_T7 to P58_T7)

    t_stat, p_ttest = ttest_rel(Tlast_values, Tfirst_values)
    w_stat, p_wilcoxon = wilcoxon(Tlast_values, Tfirst_values)

    differences['paired-t'] = t_stat
    differences['paired-p'] = p_ttest
    differences['WilCoxnRank-w'] = w_stat
    differences['WilCoxnRank-p'] = p_wilcoxon

    print(t_stat.shape, p_ttest.shape, w_stat.shape, p_wilcoxon.shape)

    row_means = differences.mean(axis=1)
    differences["avg"] = row_means
    differences['Peptidoform'] = df['Peptidoform']
    differences['Proteins'] = df['Proteins']

    file_path = os.path.join(f"./data/log_data/{filename}")
    differences.to_csv(file_path, index=True) 
        


Data Shape for 6.csv is (0, 321)
Data Shape for 6.csv after dropping 90% data (0, 321)
52 52
(0,) (0,) (0,) (0,)
Data Shape for 7.csv is (0, 321)
Data Shape for 7.csv after dropping 90% data (0, 321)
52 52
(0,) (0,) (0,) (0,)
Data Shape for 5.csv is (5, 321)
Data Shape for 5.csv after dropping 90% data (1, 321)
52 52
(1,) (1,) (1,) (1,)
Data Shape for 4.csv is (0, 321)
Data Shape for 4.csv after dropping 90% data (0, 321)
52 52
(0,) (0,) (0,) (0,)
Data Shape for 1.csv is (0, 321)
Data Shape for 1.csv after dropping 90% data (0, 321)
52 52
(0,) (0,) (0,) (0,)
Data Shape for 3.csv is (143, 321)
Data Shape for 3.csv after dropping 90% data (23, 321)
52 52
(23,) (23,) (23,) (23,)
Data Shape for 2.csv is (35, 321)
Data Shape for 2.csv after dropping 90% data (4, 321)
52 52
(4,) (4,) (4,) (4,)
Data Shape for 9.csv is (0, 321)
Data Shape for 9.csv after dropping 90% data (0, 321)
52 52
(0,) (0,) (0,) (0,)
Data Shape for 8.csv is (6, 321)
Data Shape for 8.csv after dropping 90% data (0, 321)
5